In [ ]:
# Agent Creator Demo Notebook 🤖

This notebook demonstrates the key features of the Agent Creator library, including:
- Research Agent capabilities
- Webscraper Agent functionality
- Integration between agents
- MLX-powered AI processing

Let's explore how to use these powerful tools for AI-powered research and web scraping!


In [1]:
# Import required libraries
import os
import sys
from typing import Dict, Any, List
from datetime import datetime

# Add the parent directory to Python path to import agent_creator
sys.path.append(os.path.abspath(os.path.join(os.getcwd())))

# Import our agents and utilities
from agent_creator import ResearchAgent, WebscraperAgent
from agent_creator.core.base_agent import AgentConfig, AgentTask, BaseAgent
from agent_creator.agents.webscraper_agent import ScrapingConfig, ScrapingResult
from agent_creator.agents.research_agent import ResearchResult
from agent_creator.utils.llm_interface import LLMInterface


In [ ]:
## 1. Initialize Research Agent

First, let's set up our Research Agent with appropriate configuration. 
This agent will be responsible for conducting AI-powered research tasks.


In [ ]:
# Configure and initialize the Research Agent
research_config = AgentConfig(
    name="DemoResearchAgent",
    description="AI-powered research agent for comprehensive online research",
    capabilities=[
        "web_search", "content_analysis", "citation_generation",
        "pdf_generation", "notebook_generation"
    ]
)

research_agent = ResearchAgent(research_config)
research_agent.start()  # Start the agent

print("Research Agent initialized with capabilities:", research_agent.config.capabilities)


In [ ]:
## 2. Initialize Webscraper Agent

Now, let's set up our Webscraper Agent with appropriate configuration for content extraction and web scraping tasks.


In [ ]:
# Configure and initialize the Webscraper Agent
webscraper_config = AgentConfig(
    name="DemoWebscraperAgent",
    description="Advanced web scraping agent for content extraction",
    capabilities=[
        "url_scraping", "content_extraction", "link_extraction",
        "image_extraction", "metadata_extraction", "batch_scraping"
    ]
)

# Set up scraping-specific configuration
scraping_config = ScrapingConfig(
    timeout=30,
    delay_between_requests=1.0,
    max_content_length=2000000  # 2MB
)

webscraper_agent = WebscraperAgent(webscraper_config, scraping_config)
webscraper_agent.start()  # Start the agent

print("Webscraper Agent initialized with capabilities:", webscraper_agent.config.capabilities)


In [ ]:
## 3. Connect Agents

Let's connect our Research Agent with the Webscraper Agent to enable seamless collaboration between them.


In [ ]:
# Connect the Webscraper Agent to the Research Agent
research_agent.set_webscraper_agent(webscraper_agent)

print("Agents connected successfully!")


In [ ]:
## 4. Demo: Research Task

Let's demonstrate how to use the Research Agent to conduct research on a specific topic. This will showcase the agent's ability to search, analyze, and generate insights.


In [ ]:
# Example research task
research_query = "Latest developments in MLX and machine learning on Apple Silicon"

# Create research task
research_task_id = research_agent.create_task(
    description="Research MLX and Apple Silicon developments",
    parameters={
        "type": "research",
        "query": research_query,
        "max_results": 5
    }
)

# Execute research task
research_result: ResearchResult = research_agent.run_task(research_task_id)

# Display results
print("\nResearch Results:")
print("-" * 50)
print(f"Query: {research_result.query}")
print(f"\nSummary:\n{research_result.summary}\n")

print("\nSources:")
for idx, source in enumerate(research_result.sources, 1):
    print(f"\nSource {idx}:")
    print(f"Title: {source['title']}")
    print(f"URL: {source['url']}")
    print(f"Snippet: {source['snippet'][:200]}...")

print("\nCitations:")
for citation in research_result.citations:
    print(f"- {citation}")

# Generate PDF report
pdf_task_id = research_agent.create_task(
    description="Generate PDF report from research results",
    parameters={
        "type": "generate_pdf",
        "research_result": research_result.__dict__,
        "filename": "mlx_research_report.pdf"
    }
)
pdf_path = research_agent.run_task(pdf_task_id)
print(f"\nPDF report generated: {pdf_path}")

# Generate analysis notebook
notebook_task_id = research_agent.create_task(
    description="Generate analysis notebook from research results",
    parameters={
        "type": "generate_notebook",
        "research_result": research_result.__dict__,
        "filename": "mlx_research_analysis.ipynb"
    }
)
notebook_path = research_agent.run_task(notebook_task_id)
print(f"Analysis notebook generated: {notebook_path}")


In [ ]:
## 5. Demo: Web Scraping Task

Now let's demonstrate the Webscraper Agent's capabilities by extracting content from a specific URL.


In [ ]:
# Example web scraping task
target_url = "https://ml-explore.github.io/mlx/build/html/index.html"

# Create scraping task
scraping_task_id = webscraper_agent.create_task(
    description="Scrape MLX documentation page",
    parameters={
        "type": "scrape_url",
        "url": target_url,
        "use_selenium": False  # Use requests by default
    }
)

# Execute scraping task
scraping_result: ScrapingResult = webscraper_agent.run_task(scraping_task_id)

# Display results
print("\nWeb Scraping Results:")
print("-" * 50)
print(f"URL: {scraping_result.url}")
print(f"Success: {scraping_result.success}")
print(f"\nTitle: {scraping_result.title}")
print(f"Content Preview: {scraping_result.text[:200]}...")

# Display extracted links (if any)
if scraping_result.links:
    print("\nExtracted Links (first 5):")
    for link in scraping_result.links[:5]:
        print(f"- {link}")

# Display extracted images (if any)
if scraping_result.images:
    print("\nExtracted Images (first 3):")
    for image in scraping_result.images[:3]:
        print(f"- {image}")

# Display metadata
print("\nMetadata:")
for key, value in scraping_result.metadata.items():
    print(f"{key}: {value}")

if scraping_result.error:
    print(f"\nError: {scraping_result.error}")

print(f"\nResponse Time: {scraping_result.response_time:.2f} seconds")


In [ ]:
## 6. Error Handling and Cleanup

Finally, let's demonstrate proper error handling and cleanup of our agents.


In [ ]:
try:
    # Example of handling potential errors
    invalid_url = "https://nonexistent-website-12345.com"
    error_task_id = webscraper_agent.create_task(
        description="Attempt to scrape non-existent website",
        parameters={
            "type": "scrape_url",
            "url": invalid_url
        }
    )
    result: ScrapingResult = webscraper_agent.run_task(error_task_id)
    
    if result.error:
        print(f"Expected error occurred: {result.error}")
    else:
        print("Unexpected success!")
        
except Exception as e:
    print(f"Unexpected error occurred: {str(e)}")
finally:
    # Get final task statuses
    print("\nFinal Task Statuses:")
    print("-" * 50)
    for task_id in [research_task_id, scraping_task_id, error_task_id]:
        status = webscraper_agent.get_task_status(task_id)
        print(f"Task {status['task_id']}: {status['status']}")
    
    # Cleanup
    research_agent.stop()
    webscraper_agent.stop()
    print("\nAgents cleaned up successfully!")
